<h1><center>Laboratorio 9: Los huesos de Hipócrates 🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Roberto Barceló


### **Link de repositorio de GitHub:** `http://....`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## Temas a tratar

- Creación de clasificadores de imagenes a traves de redes Fully connected y CNN.
- Uso de Dataloaders para la carga de datasets.
- Comparación de Fully Connected y red convolucional.

## Reglas:

- Fecha de entrega: 17/11/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Creación de modelos de clasificación de imágenes utilizando Pytorch.
- Creación de dataloader y aplicar transformaciones sobre el dataset.
- Comprender la diferencia entre una CNN y una Fully Connected.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones de `Pytorch`, la cual, está enfocada para proyectos de Deep Learning.


# Importamos librerias utiles 😸

Comenzamos importando librerías utiles para la ejecución del laboratorio:

In [1]:
import os
import time
from PIL import Image

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import datasets, models
from torchvision import transforms as T

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Identificando los Huesos de Hipócrates🔎

<p align="center">
  <img src="https://i.pinimg.com/originals/d8/58/66/d85866cd1cc3979f94526551addf74b4.gif" width="300">
</p>

Tras el éxito que han tenido proponiendo modelos de machine learning en trabajos anteriores, el famoso medico Hipócrates se ha contactado con ustedes para solicitarles ayuda para automatizar la identificación de radiografías de partes humanas. Para esto, les señala que le gustaría utilizar algoritmos de deep learning producto que Demócrito le señalo que resultan la mejor alternativa para la predicción de imágenes.

En su conversación con el medico usted le comenta que ha tenido algunas clases relacionadas a Deep Learning, por esto, están motivados en abordar el problema utilizando redes Fully Connected y redes convolucionales con Pytorch. Sin embargo, al anunciarle los tipos de redes que conocen, el filósofo les comenta que no había escuchado muy buenos resultados por parte de las CNN, por lo que les pide que comprueben a traves de la métrica de accuracy que tipo de redes es mejor para la tarea de identificación de radiografías. ¿Será cierto lo que dice el filósofo?, Veámoslo en un nuevo capítulo de los Laboratorios de Programación Científica para Ciencia de Datos!

## 1.1 Creación de Lista de Archivos

<p align="center">
  <img src="https://media.tenor.com/BJ-9w-MUVCMAAAAM/tis100-sad.gif" width="300">
</p>


Comience revisando de forma manual cada una de las imágenes que posee la carpeta subida a material docente. Verifique la cantidad de tipos de radiografías que se tienen y la cantidad de imágenes que dispone cada carpeta.

Revisado el contenido de las imágenes, utilice `os.listdir` para crear un `numpy.array` o un `Dataframe` que contenga las imágenes y un label que señale al tipo de radiografía a la que hace referencia la imagen. Para hacer las etiquetas codifique el tipo de imágenes en números que vayan del 0 al total de tipos de radiografías, no utilice strings para codificar las etiquetas.

**Ejemplo de Estructura:**

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-0lax">image_path</th>
    <th class="tg-0lax">label</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">image1</td>
    <td class="tg-0lax">1</td>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">image2</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">image3</td>
    <td class="tg-0lax">2</td>
  </tr>
  <tr>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">image4</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">image5</td>
    <td class="tg-0lax">4</td>
  </tr>
</tbody>
</table>

In [ ]:
#!unzip '/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/2022111011141005A0888D541A93__Medical-MNIST.zip' -d '/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/'

In [4]:
# Código Aquí
df_labels = pd.DataFrame(columns=['image_path','label'])
for i, category in enumerate(os.listdir("/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST")):
    data_carpeta = []
    for image in os.listdir("/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST/"+category):
      #image_path = '/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST/'+category+'/'+image
      image_path = category+'/'+image
      label = i
      data_carpeta.append((image_path,label))
    df_carpeta = pd.DataFrame(data_carpeta,columns=['image_path','label'])
    df_labels = pd.concat([df_labels,df_carpeta], ignore_index=True)
    

In [5]:
df_labels.head(10)

,image_path,label
0,CXR/006335.jpeg,0
1,CXR/004607.jpeg,0
2,CXR/006184.jpeg,0
3,CXR/000754.jpeg,0
4,CXR/007814.jpeg,0
5,CXR/002043.jpeg,0
6,CXR/000099.jpeg,0
7,CXR/007783.jpeg,0
8,CXR/008309.jpeg,0
9,CXR/004973.jpeg,0


## 1.2 Creación de Dataset

Tomando en cuenta la estructura de datos desarrollada en el punto 1.1, construya la clase `MedicalDataset()` que cumpla los siguientes puntos:

- [X] Poseer un `__init__` en el que se almacene `estructura` creada en 1.1, la `raiz` de la carpeta y una función que permita transformar el dataset (de esto no se preocupe mucho, ya que solamente debe almacenar una función en el atributo).
- [X] La clase debe ser capaz de entregar la cantidad de elementos a traves de `__len__`.
- [X] Debe poseer el método `__getitem__` que retorne una tupla con la imagen y su correspondiente etiqueta.

In [6]:
# Código Aquí
class MedicalDataset(Dataset):
    def __init__(self, df_estructura, raiz_carpeta, transform = None):
        self.estructura = df_estructura
        self.raiz = raiz_carpeta
        self.transform = transform
    
    def __getitem__(self, id):
        
        # Un poco de ayuda para cargar la imagen
        img_path = os.path.join(self.raiz, self.estructura.iloc[id,0])
        image = Image.open(img_path).convert('RGB')
        label = self.estructura.iloc[id,1]
        
        # Auida para realizar la transformación
        if self.transform:
            image = self.transform(image)
        
        return (image,label)
    
    def __len__(self):
        return len(self.estructura)

## 1.3 Prueba del MedicalDataset

Con la clase construida en el punto 1.2, verifique su funcionamiento cargando el dataset y realizando las transformaciones que entrega la función `transform_image`. Compruebe a través de un ejemplo las transformaciones aplicadas en la imagen, comentando la función que cumple `MedicalDataset` y si es posible observar todas las transformaciones aplicadas con la función `transform_image`.

- [X] Probar la clase MedicalDataset y aplicando una transformación de "train"
- [X] Plotear un ejemplo del MedicalDataset.

**Función para transformar las imagenes:**

In [7]:
def transform_image(stage = None):
    
    if stage == "train":
        Tr_img = T.Compose([T.Resize(size = (64,64)),
                T.RandomRotation(degrees = (-20,+20)),
                T.ToTensor()])
        
    elif stage == "test" or stage == "val":
        Tr_img = T.Compose([T.Resize(size = (64,64)), T.ToTensor()]) 

    return Tr_img

**Código para obtener un ejemplo:**

In [8]:
# Prueba del dataset
dataset = MedicalDataset(df_estructura = df_labels, raiz_carpeta = "/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST/", transform = transform_image("train"))

In [9]:
data_test= iter(dataset)
example_image = next(data_test)

# Utilice plotly para plotear un ejemplo
#example_image[0][0,:,:].shape
img_ex = px.imshow(example_image[0][0,:,:])
img_ex.show()

In [10]:
example_image[0][:,:,:].shape

torch.Size([3, 64, 64])

> Comente que realiza la clase construida y las transformaciones aplicadas.

## 1.4 Creación de Clasificadores

<p align="center">
  <img src="https://149695847.v2.pressablecdn.com/wp-content/uploads/2018/01/conv-full-layer.gif" width="300">
</p>

A continuación, deben construir tres clasificadores con los que deberán verificar cuál de las arquitecturas posee un mejor desempeño para la tarea de clasificación de imágenes. Para la construcción considere los siguientes puntos:

- [ ] Señale cual es el objetivo del `forward` en este tipo de redes, sea breve para su explicación.
- [ ] Construir una red Fully Connected para solucionar el problema de clasificación. Para esta parte se le aconseja que rellene el esqueleto dispuesto más abajo y que lleva el nombre de `FCClassifier`, en el deberá rellenar con la dimensión de las capas ocultas y verificar cual será el tamaño de la entrada.
- [ ] Construya una red convolucional **simple** (no más de una capa convolucional) para la tarea de clasificación de imágenes, para esto basen su código en la clase del día `09-11-2022`.
- [ ] Crear una red convolucional más compleja. Para esta parte tienen completa libertad en la construcción de su red, lo único que debe cumplir es que sea convolucional.

**Esqueletos Propuestos:**

In [11]:
class FCClassifier(nn.Module):
    def __init__(self, in_channels, hidden_size, num_classes):
        super(FCClassifier, self).__init__()
        #self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(in_channels, hidden_size)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        #out = self.flatten(x)
        out = x.view(x.size(0), -1)
        out = self.lin1(out)
        out = self.relu(out)
        out = self.lin2(out)

        return out

In [12]:
class CNNClassifier1(nn.Module):
    def __init__(self, in_channels,num_classes):
        super(CNNClassifier1, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=3, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc1 = nn.Linear(in_features=6144, out_features=40)
        self.relu_1 = nn.ReLU()
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=40, out_features=32)
        self.relu_2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=32, out_features=num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu_1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.relu_2(out)
        out = self.fc3(out)
        # noten que al optimizar con CrossEntropyLoss no necesitamos que la salida sea probabilidad.
        # es decir, no necesitamos poner un softmax previo a la salida
        return out

class CNNClassifier2(nn.Module):
    def __init__(self, in_channels,num_classes):
        super(CNNClassifier2, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc1 = nn.Linear(in_features=4096, out_features=64)
        self.relu_1 = nn.ReLU()
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.relu_2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=32, out_features=num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu_1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.relu_2(out)
        out = self.fc3(out)
        # noten que al optimizar con CrossEntropyLoss no necesitamos que la salida sea probabilidad.
        # es decir, no necesitamos poner un softmax previo a la salida
        return out

## 1.5 Separando Datos para el Entrenamiento

<p align="center">
  <img src="https://c.tenor.com/Esn7Jif-ZLQAAAAC/separate-square.gif" width="200">
</p>

Utilizando un Holdout a su gusto, separe los datos en un conjunto de entrenamiento y de testing. Aplique las transformaciones correspondientes usando `transform_image` para cada conjunto de datos y utilice `torch.utils.data.DataLoader` para crear un objeto iterable del dataset.

- [ ] Definir el Holdout a utilizar.
- [ ] Separar los datos en un conjunto de entrenamiento y prueba.
- [ ] Aplicar las transformaciones correspondientes en cada uno de los dataset.
- [ ] Utilizar `DataLoader` de pytorch sobre los dataset.

In [13]:
len(dataset)

5948

In [14]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_labels, test_size = 0.2, random_state=123)

In [15]:
# Separar Datos de Entrenamiento
train_data = MedicalDataset(df_estructura = train_df, raiz_carpeta = "/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST/", transform = transform_image("train"))
test_data  = MedicalDataset(df_estructura = test_df, raiz_carpeta = "/content/drive/MyDrive/MDS7202_LPCPCD/Lab 9/Medical-MNIST/", transform = transform_image("test"))

In [16]:
# Uso de torch.utils.data.DataLoader
train_loader = DataLoader(train_data, batch_size=200, shuffle=True)
test_loader = DataLoader(test_data, batch_size=200,shuffle=True)

## 1.6 Creación de Funciones de Entrenamiento y Evaluación

<p align="center">
  <img src="https://www.researchgate.net/publication/319535615/figure/fig3/AS:536187598065664@1504848493070/A-typical-convolutional-neural-network-CNN-Architecture-for-Medical-Image-Classification.png" width="500">
</p>


Ya construido todas las funciones y clases necesarias llego el momento más importante... probar la red. Para esta sección, ustedes deberán ser capaces de definir los hiperparámetros de la red, definir las funciones de perdida a utilizar, señalar el optimizador a usar y finalmente crear sus funciones para el entrenamiento y prueba. Para realizar esta parte más estructurada, seguir los siguientes puntos de forma secuencial:

- [ ] Especifique los Hiperparámetros de las 3 redes. Para esta parte sea claro de su elección y señale el porqué de sus elecciones (o sea justifique el setting de sus hiperparámetros).
- [ ] Defina los modelos a utilizar, el optimizador que utilizará para el modelo y señale la función de perdida que utilizará.
- [ ] Explique de forma breve la función que cumplen los pasos `Backward` y `Descenso del gradiente` en una red neuronal.
- [ ] Cree una función llamado `train` que entrene a los clasificadores. Para esto, recuerde que estos modelos suelen utilizar un número de épocas, por lo que deberá generar un proceso iterativo de entrenamiento. Es importante que su función imprima las `loss` obtenidas por el modelo en cada época (si gusta puede almacenar estas losses en una lista para luego graficarlas y comparar).
- [ ] Diseñe una función para evaluar el desempeño de las redes. Para evaluar las redes utilice solamente la métrica accuracy (para esto se le recomienda comparar la predicción con el ground truth)

In [28]:
# Especificar hyperparámetros de las redes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channels_fc = 3*64*64
hidden_size_fc = 32
in_channels_cnn = 3
num_classes = 6
lr = 0.001
batch_size = 200
n_epochs = 5

In [18]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')

In [29]:
# Red 1
model_fc = FCClassifier(in_channels_fc,hidden_size_fc,num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fc.parameters(),lr=lr)

In [30]:
count_parameters(model_fc)

  393216
      32
     192
       6
________
  393446


In [21]:
# Red 2
model_cnn_1 = CNNClassifier1(in_channels_cnn,num_classes).to(device)

# Loss and optimizer
criterion_1 = nn.CrossEntropyLoss()
optimizer_1 = optim.Adam(model_cnn_1.parameters(),lr=lr)

In [22]:
count_parameters(model_cnn_1)

     162
       6
       6
       6
  245760
      40
    1280
      32
     192
       6
________
  247490


In [23]:
# Red 3
model_cnn_2 = CNNClassifier2(in_channels_cnn,num_classes).to(device)

# Loss and optimizer
criterion_2 = nn.CrossEntropyLoss()
optimizer_2 = optim.Adam(model_cnn_2.parameters(),lr=lr)

In [24]:
count_parameters(model_cnn_2)

     432
      16
      16
      16
    4608
      32
      32
      32
   18432
      64
      64
      64
  262144
      64
    2048
      32
     192
       6
________
  288294


In [25]:
model_cnn_1

CNNClassifier1(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=6144, out_features=40, bias=True)
  (relu_1): ReLU()
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=40, out_features=32, bias=True)
  (relu_2): ReLU()
  (fc3): Linear(in_features=32, out_features=6, bias=True)
)

In [26]:
# train
def train(model, train_loader, num_epochs, criterion, optimizer):
    for e in range(n_epochs):
        for i, (data, label) in enumerate(train_loader):
            data = data.to(device=device)
            targets = label.to(device=device)
            
            #Forward
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            #Backward
            optimizer.zero_grad()
            loss.backward()
            
            # Descenso del gradiente
            optimizer.step()

            if i%200==0:
              print(f"epoch: {e} loss: {loss} batch: {i}")
            # testeo del modelo en proceso de entrenamiento
        print(f"epoch: {e} loss: {loss}")
# Evaluate
def evaluate(test_loader, model):
    correctas = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for X,y in test_loader:
          X = X.to(device=device)
          y = y.to(device=device)

          scores = model(X)

          pred = scores.max(1)[1]
          correctas += (pred == y).sum()
          total += pred.size(0)
        print(f"Accuracy: {correctas/total*100}%")

## 1.7 Comparación de Resultados

<p align="center">
  <img src="https://media2.giphy.com/media/icJA0VF7ntoEL18Jez/giphy.gif"  width="200">
</p>

Construidas las funciones de entrenamiento y evaluación, entrene a las redes que construyo y compare los resultados obtenidos con todas las redes señalando cual posee mejor rendimiento. Comente una diferencia entre las redes Fully Connected y CNN podría generar un mejor desempeño en una u otra en la tarea de clasificación de imágenes.

- [ ] Entrenar las redes.
- [ ] Evaluar las redes.
- [ ] Comentar los resultados obtenidos y visualizar si existe una diferencia significativa en el rendimiento debido a la naturaleza de la red.

In [31]:
# Entrenamiento red 1
train(model_fc, train_loader, n_epochs, criterion, optimizer)

epoch: 0 loss: 1.774779200553894 batch: 0
epoch: 0 loss: 0.4188772737979889
epoch: 1 loss: 0.4124292731285095 batch: 0
epoch: 1 loss: 0.3398815095424652
epoch: 2 loss: 0.2963653802871704 batch: 0
epoch: 2 loss: 0.20721213519573212
epoch: 3 loss: 0.24453258514404297 batch: 0
epoch: 3 loss: 0.28069525957107544
epoch: 4 loss: 0.16786926984786987 batch: 0
epoch: 4 loss: 0.14956210553646088


In [32]:
# Testing red 1
evaluate(test_loader, model_fc)

Accuracy: 94.87395477294922%


In [33]:
# Entrenamiento red 2
train(model_cnn_1, train_loader, n_epochs, criterion_1, optimizer_1)

epoch: 0 loss: 1.7974787950515747 batch: 0
epoch: 0 loss: 0.34788578748703003
epoch: 1 loss: 0.44516944885253906 batch: 0
epoch: 1 loss: 0.2630845904350281
epoch: 2 loss: 0.17768435180187225 batch: 0
epoch: 2 loss: 0.11434551328420639
epoch: 3 loss: 0.08117161691188812 batch: 0
epoch: 3 loss: 0.151793971657753
epoch: 4 loss: 0.06538262218236923 batch: 0
epoch: 4 loss: 0.12267723679542542


In [34]:
# testing red 2
evaluate(test_loader, model_cnn_1)

Accuracy: 94.2016830444336%


In [35]:
# Entrenamiento red 3
train(model_cnn_2, train_loader, n_epochs, criterion_2, optimizer_2)

epoch: 0 loss: 1.7746678590774536 batch: 0
epoch: 0 loss: 0.07432598620653152
epoch: 1 loss: 0.06431233137845993 batch: 0
epoch: 1 loss: 0.014405625872313976
epoch: 2 loss: 0.06072736904025078 batch: 0
epoch: 2 loss: 0.017537763342261314
epoch: 3 loss: 0.011610065586864948 batch: 0
epoch: 3 loss: 0.011938374489545822
epoch: 4 loss: 0.00530437333509326 batch: 0
epoch: 4 loss: 0.009555329568684101


In [36]:
# testing red 3
evaluate(test_loader, model_cnn_2)

Accuracy: 99.7479019165039%


> Comente los resultados

# Conclusión

Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/vKSR-ZakVMIAAAAC/pochitadancing-pochita.gif">
</p>